## Описание колонок

`Rank` - Общий рейтинг продаж

`Name` - Название игры

`Platform` - Платформа, на которой выпущена игра (например, PC, PS4 и т.д.)

`Year` - Год выпуска игры

`Genre` - Жанр игры

`Publisher` - Издатель игры

`NA_Sales` - Продажи в Северной Америке (в миллионах)

`EU_Sales` - Продажи в Европе (в миллионах)

`JP_Sales` - Продажи в Японии (в миллионах)

`Other_Sales` - Продажи в остальном мире (в миллионах)

`Global_Sales` - Общемировые продажи.

В Pandas есть две основные высокоуровневые структуры данных – DataFrame и Series.

Series – это одномерная маркированная структура данных, состоящая из индексов и соответствующих значений. В качестве меток могут выступать числа, даты, временные интервалы и строки. Series можно рассматривать как упорядоченный словарь

DataFrame – это двумерная табличная структура данных. DataFrame можно сравнить с таблицей в реляционной базе данных

Pandas с нуля: https://pythonist.ru/polnoe-rukovodstvo-po-pandas-dlya-nachinayushhih/#series-and-dataframes

In [1]:
import pandas as pd

# Загрузить Dataset, создать Data frame
df = pd.read_csv('../resources/datasets/vgsales.csv')  # ваш код здесь

# после того датасет корректно загрузился, можно посмотреть на данные глазами
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


1. Количество уникальных игр в датасете

In [8]:
# Вывод количества уникальных (по наименованию) игр
print(f'Count of unique games in dataset: {len(df.Name.unique())}')


Count of unique games in dataset: 11493


2. Всего продано копий игр в Японии

In [12]:
print(f'Всего продано копий игр в Японии: {int(df.JP_Sales.sum() * 1E6)} шт')

Всего продано копий игр в Японии: 1291020000 шт


3. Три самых популярнных жанра в Северной Америке

In [17]:
print(f'Top-3 most popular genres in North America: {list(df.sort_values(by='NA_Sales', ascending=False).head(3).Genre)}')

Top-3 most popular genres in North America: ['Sports', 'Platform', 'Shooter']


4. Самая популярная платформа для выпуска игр:
   - В 2000 году
   - В 2015 году

In [3]:
df_copy = df.copy()
# Сгруппировать по году + платформе. Просуммировать продажи игр, выпущенных в году X на платформе Y
# Результат подсчета записать в столбец с наименованием GSales
gdata = df_copy.groupby(by=['Year', 'Platform'])['Global_Sales'].sum().to_frame('GSales').reset_index()

# Отфильтровать таблицу по значению в поле Year
top_platform_2000 = gdata[gdata['Year'] == 2000.0]
top_platform_2015 = gdata[gdata['Year'] == 2015.0]

# top_platform_2000['GSales'].idxmax() возвращает индекс строки с максимальным значением в поле GSales (первое вхождение)
# После чего от этой строки получить значение в поле Platform. Метод loc позволяет получить значение (строку) по индексу (грубо говоря id строки)
top_platform_2000 = top_platform_2000.loc[top_platform_2000['GSales'].idxmax()].Platform
top_platform_2015 = top_platform_2015.loc[top_platform_2015['GSales'].idxmax()].Platform

print(f'Most popular game platform in 2000s: {top_platform_2000}')
print(f'Most popular platform in 2015: {top_platform_2015}')
# grouped_data

Most popular game platform in 2000s: PS
Most popular platform in 2015: PS4


5. Издатель, выпустивший наибольшее число игр в период с 2012 по 2015 годы

In [4]:
# Фильтрация записей по периоду
df_filtered = df_copy[(df_copy['Year'] >= 2012.0) & (df_copy['Year'] <= 2015.0)]
# Подсчет количество раз, которое встречается каждый издатель в таблице и записать в поле Releases
df_filtered = df_filtered['Publisher'].value_counts().to_frame('Releases').reset_index()
# Вывод значения поля Publisher в самой первой строке, т.к. value_counts() упорядочивает значения по убыванию
print(f'The publisher with the most games published in 2012-2015: {df_filtered['Publisher'].loc[0]}')

The publisher with the most games published in 2012-2015: Namco Bandai Games


6. Какой процент от общих продаж игр в жанре "Спорт" составляют продажи в Европе?

In [21]:
# Количество копий игр в жанре "Спорт", проданных:
# EU_Sales - в Европе, Global_Sales - по всему миру
df_stats = df[df['Genre'] == 'Sports'][['EU_Sales', 'Global_Sales']].sum()
svg_eu_sales = round(df_stats['EU_Sales'], 2)
svg_gl_sales = round(df_stats['Global_Sales'], 2)

print(f'Local copies count (Europe) in Sports genre: {svg_eu_sales}')
print(f'Total copies count (Global Sales) in Sports genre: {svg_gl_sales}')
print(f'Sports genre sale percent: {round((svg_eu_sales/svg_gl_sales)*100, 2)}%')

Local copies count (Europe) in Sports genre: 376.85
Total copies count (Global Sales) in Sports genre: 1330.93
Sports genre sale percent: 28.31%


6. (old) Какой процент от продаж в Европе составляют игры в жанре "Спорт"

In [22]:
# Количество игр в жанре Спорт, проданных в Европе
sport_games = round(df[(df['Genre'] == 'Sports')]['EU_Sales'].sum(), 2)

# Общее количество игр, проданных в Европе (все жанры)
all_games = round(df['EU_Sales'].sum(), 2)

print(f'Sport copies count: {sport_games}')
print(f'Total copies count: {all_games}')
print(f'Sports genre sale percent: {round((sport_games/all_games)*100, 2)}%')

Sport copies count: 376.85
Total copies count: 2434.13
Sports genre sale percent: 15.48%
